In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
import json
import os
from tqdm import tqdm

In [2]:
md = pd.read_csv('/Users/mainoahmuna/Google Drive/My Drive/Team-Fermata-Energy/processed_data/md_encoded.csv')
PATH = '/Users/mainoahmuna/Google Drive/My Drive/Team-Fermata-Energy/processed_data/processed_weather_load_w_timestamp'
json_file = '../data/subset20_data.json'

In [3]:
def load_building_ids(json_file_path):
    """
    Load train and test building IDs from a JSON file.

    Parameters:
        json_file (str): Path to the JSON file containing building IDs.

    Returns:
        tuple: Two lists, one for training building IDs and one for testing building IDs.
    """
    with open(json_file_path, 'r') as file:
        data = json.load(file)
    train_bldg_ids = data.get("train_bldg_ids", [])
    test_bldg_ids = data.get("test_bldg_ids", [])
    return train_bldg_ids, test_bldg_ids

In [4]:
def smape(actual, predicted):
    """
    Calculate SMAPE (Symmetric Mean Absolute Percentage Error) between actual and predicted values.
    """
    actual, predicted = np.array(actual), np.array(predicted)
    denominator = np.abs(actual) + np.abs(predicted)
    diff = np.abs(actual - predicted) / denominator
    diff[denominator == 0] = 0.0  # Handle division by zero
    return 200 * np.mean(diff)

In [5]:
def create_df(df_load, md):
    """
    Create Y and X variables for linear regression model.

    Parameters:
        df_load (pandas.DataFrame): DataFrame containing load data.

    Returns:
        tuple: Tuple containing Y and X variables.  
    """
    for i in range(1, 97):
        df_load[f"shift_{i}"] = df_load["out.electricity.total.energy_consumption"].shift(i)

    df_load = df_load.dropna()

    # Merge load data with metadata based on `building_id`
    merged_df = df_load.merge(md, on='bldg_id', how='left')

    merged_df = merged_df.drop(['bldg_id'], axis=1)
    
    return merged_df

In [6]:
def train_sarimax_model(df, target_column, order=(1, 1, 1), seasonal_order=(0, 1, 1, 96)):
    """
    Train a SARIMAX model on the provided DataFrame.

    Parameters:
        df (pandas.DataFrame): DataFrame containing the time series data.
        target_column (str): Column name of the target variable.
        order (tuple): SARIMA order (p, d, q).
        seasonal_order (tuple): Seasonal order (P, D, Q, s).
    
    Returns:
        SARIMAXResults: The trained SARIMAX model.
    """
    model = SARIMAX(df[target_column], order=order, seasonal_order=seasonal_order, enforce_stationarity=False)
    results = model.fit(disp=False)
    return results

In [7]:
def predict_sarimax_model(model, steps, exog=None):
    """
    Use a trained SARIMAX model to predict future values.

    Parameters:
        model (SARIMAXResults): Trained SARIMAX model.
        steps (int): Number of steps to predict.
        exog (pandas.DataFrame, optional): Exogenous variables for forecasting.

    Returns:
        pandas.DataFrame: Predictions with confidence intervals.
    """
    forecast = model.get_forecast(steps=steps, exog=exog)
    forecast_df = forecast.conf_int()
    forecast_df['mean'] = forecast.predicted_mean
    return forecast_df

In [8]:
def evaluate_sarimax_model(model, train_data, test_data, steps, exog_train=None, exog_test=None):
    """
    Evaluate SARIMAX model by calculating SMAPE for training and testing data.
    
    Parameters:
        model (SARIMAXResults): Trained SARIMAX model.
        train_data (pandas.Series): Training data (endogenous variable).
        test_data (pandas.Series): Testing data (endogenous variable).
        steps (int): Number of steps to predict in testing data.
        exog_train (pandas.DataFrame, optional): Exogenous variables for training.
        exog_test (pandas.DataFrame, optional): Exogenous variables for testing.
    
    Returns:
        tuple: SMAPE for training and testing data.
    """
    # Get in-sample predictions for training data
    train_forecast = model.get_prediction(start=train_data.index[0], end=train_data.index[-1], exog=exog_train)
    train_predicted = train_forecast.predicted_mean
    
    # Calculate SMAPE for training
    smape_train = smape(train_data, train_predicted)
    
    # Get out-of-sample predictions for testing data
    test_forecast = model.get_forecast(steps=steps, exog=exog_test)
    test_predicted = test_forecast.predicted_mean
    
    # Align predictions with actual test data
    test_predicted_index = test_data.index[:steps]
    test_predicted = test_predicted.loc[test_predicted_index]
    
    # Calculate SMAPE for testing
    smape_test = smape(test_data.loc[test_predicted_index], test_predicted)
    
    return smape_train, smape_test

In [9]:
df_test = pd.read_csv('/Users/mainoahmuna/Google Drive/My Drive/Team-Fermata-Energy/processed_data/processed_weather_load_w_timestamp/32.csv')

In [10]:
def load_data_for_building(filename, directory):
    """
    Load the data for a specific building from CSV.
    """
    try:
        df = pd.read_csv(f"{directory}/{filename}")
        return df
    except Exception as e:
        print(f"Error loading file {filename}: {e}")
        return None

In [11]:
def train_sarimax_model_with_downsampling(directory, train_bldg_ids, test_bldg_ids, target_column='out.electricity.total.energy_consumption', steps=96, chunk_size=1000, downsample_rate=4):
    """
    Train and test the SARIMAX model using downsampled data from building CSV files.

    Parameters:
        directory (str): Path to the directory containing building CSV files.
        train_bldg_ids (list): List of building IDs (file names) for training.
        test_bldg_ids (list): List of building IDs (file names) for testing.
        target_column (str): The name of the target column.
        steps (int): Number of steps to predict for each test.
        chunk_size (int): Number of rows to process in each chunk.
        downsample_rate (int): Factor by which to downsample the training data.

    Returns:
        dict: Results containing trained models, predictions, SMAPE scores, and visualizable results.
    """
    smape_train_list = []
    smape_test_list = []
    detailed_results = []

    # Training phase
    for filename in tqdm(train_bldg_ids, desc="Training on buildings", unit="file"):
        try:
            train_df = load_data_for_building(filename, directory)
            if train_df is None or target_column not in train_df:
                print(f"Skipping {filename} due to missing data.")
                continue

            train_df = train_df.sort_index()  # Ensure data is sorted by timestamp

            # Downsample the training data
            train_df = train_df.iloc[::downsample_rate, :]  # Sample every Nth row

            # Handle missing values
            if train_df[target_column].isnull().any():
                print(f"Missing values in {filename}, skipping.")
                continue

            # Fit the SARIMAX model
            model = SARIMAX(
                train_df[target_column],
                order=(1, 1, 1),  # Adjust these parameters if necessary
                seasonal_order=(1, 1, 1, 24 // downsample_rate),  # Seasonal frequency adjusted for downsampling
                initialization='approximate_diffuse'
            )
            model_fit = model.fit(disp=False)

            # Calculate training SMAPE
            smape_train = smape(train_df[target_column], model_fit.fittedvalues)
            smape_train_list.append(smape_train)

        except Exception as e:
            print(f"Error processing file {filename} during training: {e}")
            continue

    # Testing phase
    for filename in tqdm(test_bldg_ids, desc="Testing on buildings", unit="file"):
        try:
            test_df = load_data_for_building(filename, directory)
            if test_df is None or target_column not in test_df:
                print(f"Skipping {filename} due to missing data.")
                continue

            test_df = test_df.sort_index()  # Ensure data is sorted by timestamp

            # Handle missing values
            if test_df[target_column].isnull().any():
                print(f"Missing values in {filename}, skipping.")
                continue

            # Fit the SARIMAX model on test data
            model = SARIMAX(
                test_df[target_column],
                order=(1, 1, 1),
                seasonal_order=(1, 1, 1, 24),
                initialization='approximate_diffuse'
            )
            model_fit = model.fit(disp=False)

            # Forecast
            forecast = model_fit.forecast(steps=steps)

            if len(test_df[target_column]) < steps:
                print(f"Not enough data in {filename} for forecasting {steps} steps.")
                continue

            # Compute SMAPE
            smape_test = smape(test_df[target_column][-steps:], forecast)
            smape_test_list.append(smape_test)

            # Detailed results
            detailed_results.append({
                'test_building': filename,
                'smape_train': smape_train_list[-1],
                'smape_test': smape_test,
                'predictions': forecast.tolist(),
                'actual': test_df[target_column][-steps:].tolist()
            })

        except Exception as e:
            print(f"Error processing file {filename} during testing: {e}")
            continue

    # Return results
    return smape_train_list, smape_test_list, detailed_results

In [12]:
train_id, test_id = load_building_ids(json_file)

In [13]:
smape_train_list, smape_test_list, detailed_results = train_sarimax_model_with_downsampling(PATH, train_id, test_id, downsample_rate=10)

Training on buildings:   0%|          | 0/5120 [00:00<?, ?file/s]

Training on buildings:   0%|          | 9/5120 [00:08<1:14:16,  1.15file/s]/Users/mainoahmuna/Downloads/Projects/FermataEnergy_BTTStudio/venv/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
Training on buildings:   0%|          | 10/5120 [00:10<1:25:35,  1.00s/file]/Users/mainoahmuna/Downloads/Projects/FermataEnergy_BTTStudio/venv/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
Training on buildings:   0%|          | 12/5120 [00:12<1:28:44,  1.04s/file]/Users/mainoahmuna/Downloads/Projects/FermataEnergy_BTTStudio/venv/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as s

In [14]:
print(smape_test_list)

[np.float64(67.14970392752893), np.float64(152.72987722311137), np.float64(32.73212331914938), np.float64(40.50237499997685), np.float64(59.735488693893714), np.float64(24.529144589385265), np.float64(48.037476864562436), np.float64(48.19153896172836), np.float64(38.99034504891009), np.float64(39.12095785676337), np.float64(42.7604926613802), np.float64(42.317365334452525), np.float64(12.720186449019167), np.float64(34.177894059490285), np.float64(159.3762889314663), np.float64(41.58203629645884), np.float64(83.0171870652688), np.float64(92.19277583308396), np.float64(84.03405649413592), np.float64(12.090599807339315), np.float64(16.52375946824122), np.float64(16.807082881668766), np.float64(80.28596195473257), np.float64(32.13020928157438), np.float64(19.004189565224593), np.float64(36.65488217991962), np.float64(127.75123612031017), np.float64(96.14313682778922), np.float64(103.2969119301069), np.float64(45.59763660530371), np.float64(41.19385268712641), np.float64(28.6028743533237),

In [15]:
avg_smape_train = sum(smape_train_list) / len(smape_train_list)
avg_smape_test = sum(smape_test_list) / len(smape_test_list)

In [16]:
print(avg_smape_train)
print(avg_smape_test)

30.22995662582782
54.77080645501262
